# Hi-Res vs. Coarse Benguela
A quick movie comparison of the high-resolution hindcast Benguela Current versus the low-resolution CESM1-LENS Benguela Current. 

In [1]:
import glob

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
plt.style.use('ocn-clim')
%matplotlib inline
import cmocean.cm as cm

from mpl_toolkits.basemap import Basemap

In [2]:
lat1 = -30
lat2 = -15
lon1 = 0
lon2 = 25

# Functions

In [3]:
def find_indices(latGrid, lonGrid, latPoint, lonPoint):
    dx = lonGrid - lonPoint
    dy = latGrid - latPoint
    reducedGrid = abs(dx) + abs(dy)
    min_ix = np.nanargmin(reducedGrid)
    i, j = np.unravel_index(min_ix, reducedGrid.shape)
    return i, j

# If the data has already been processed, just load the netcdf files here and skip the next two chunks

In [4]:
hires_dir = '/glade/scratch/rbrady/HI_RES/2003_50m_WVEL_Benguela.nc'
ds_hi = xr.open_dataset(hires_dir)


# Extract Benguela for Low Resolution
It looks like I currently only have global fields saved out and processed for WVEL over a fixed Landschuetzer timeline. When I have more time, I will run the same analyses on WVEL. My presentation is tomorrow that I am making this for, so obviously no time.

In [ ]:
inDir ='/glade/p/work/rbrady/EBUS_BGC_Variability/global_fields/'
filename = 'WVEL-ensemble-landschuetzer-mean-global.nc'
# This represents the global 50m WVEL mean from 1982-2011
ds_lens = xr.open_dataset(inDir + filename)

In [ ]:
lat = ds_lens['TLAT'].values
lon = ds_lens['TLONG'].values
a, c = find_indices(lat, lon, lat1, lon1)
b, d = find_indices(lat, lon, lat2, lon2)

In [ ]:
ds_lens = ds_lens['WVEL'].isel(nlat=slice(a, b), nlon=slice(c, d))

# Extract Benguela for High Resolution

In [ ]:
fileDir = '/glade/p/cmia0001/TL319_t12/archive/g.e20.G.TL319_t12.GIAF.005/ocn/hist/'
fileName = 'g.e20.G.TL319_t12.GIAF.005.pop.h.2003-*.nc'
# After getting a snapshot going, will glob in all of 2003 for a loop.

In [ ]:
ds_hi = xr.open_mfdataset(fileDir + fileName, decode_times=False)

In [ ]:
ds_hi = ds_hi['WVEL']
ds_hi = ds_hi.squeeze()
ds_hi = ds_hi[:, 5] # 50m depth

In [ ]:
timeRef = pd.date_range('2003-01', '2004-01', freq='M')
ds_hi['time'] = timeRef

In [ ]:
lat = ds_hi['TLAT'].values
lon = ds_hi['TLONG'].values

In [ ]:
a, c = find_indices(lat, lon, lat1, lon1)
b, d = find_indices(lat, lon, lat2, lon2)

In [ ]:
ds_hi = ds_hi.isel(nlat=slice(a,b), nlon=slice(c, d))

In [ ]:
ds_hi = ds_hi * (60 * 60 * 24) * (1/100) # Convert to m/day

In [ ]:
ds_hi.to_netcdf('/glade/scratch/rbrady/HI_RES/2003_50m_WVEL_Benguela.nc')

# Visualize

In [5]:
def setup_map(ax):
    m = Basemap(projection='eqdc',
               lat_0=-20, lon_0=10,
               llcrnrlat=-40, llcrnrlon=0,
               urcrnrlat=-10, urcrnrlon=25,
               resolution='c')
    m.fillcontinents(color='k')
    return m

In [6]:
# Set up data
lon = ds_hi['TLONG'].values
lat = ds_hi['TLAT'].values
data = ds_hi['WVEL'].values
data = np.ma.array(data, mask=np.isnan(data))

In [7]:
months = ['January', 'February', 'March', 'April',
          'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']
nums = ['001','002','003','004','005','006',
        '007','008','009','010','011','012']

In [8]:
for idx in np.arange(0, 12, 1):
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(111)
    m = setup_map(ax)
    m.pcolor(lon, lat, data[idx], vmin=-3, vmax=3, latlon=True, cmap=cm.balance)
    m.colorbar(extend='both')
    ax.set_title(months[idx] + ' 2003 Vertical Velocity [m day$^{-1}$]')
    plt.savefig('benguela_hires_2003_wvel_' + nums[idx] + '.png', transparent=False,
                bbox_inches='tight', pad_inches=1, dpi=1000)
    plt.clf()